In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score
from sklearn.ensemble import RandomForestRegressor


In [11]:
# Cargar los datasets (ajusta las rutas a tus archivos)
job_postings = pd.read_csv(r"job_postings.csv")
locations = pd.read_csv(r"locations.csv")
position_types = pd.read_csv(r"position_types.csv")
seniority_levels = pd.read_csv(r"seniority_levels.csv")
industry_type = pd.read_csv(r"industry_type.csv")

In [16]:
import pandas as pd

df=industry_type.copy()

def map_industry_to_category(industry):
    # Diccionario de mapeo (expandir según sea necesario)
    mapping = {
    # Tecnología
    "desarrollo de software": "Technology",
    "software empresarial y soluciones de red": "Technology",
    "internet y Servicios web": "Technology",
    "soporte informático": "Technology",
    "sesarrollo de equipos informáticos": "Technology",
    "videojuegos": "Technology",
    "computer hardware & software": "Technology",
    "it Services": "Technology",

    # Finanzas
    "aseguradoras y sociedades de inversión": "Finance",
    "Banca y Servicios de crédito": "Finance",
    "Compañías de seguros": "Finance",
    "Procesamiento de transacciones financieras": "Finance",
    "Gestión de activos e Inversiones": "Finance",
    "Banks & Credit Unions": "Finance",
    "Investment Banking & Asset Management": "Finance",
    "Lending": "Finance",
    # ... (agregar más categorías relacionadas con finanzas)

    # Salud
    "Hospitales y Centros de salud": "Healthcare",
    "Hospitales y servicios de atención médica": "Healthcare",
    "Biotecnología e industria farmacéutica": "Biotecnology",
    "Health Care Services & Hospitals": "Healthcare",
    "Health Care Products Manufacturing": "Healthcare",
    "Farmacias y tiendas de productos para la salud": "Pharmaceutical",
    
    # Manufactura
    "Fabricación de productos químicos": "Manufacturing",
    "Fabricación de productos de consumo": "Manufacturing",
    "Fabricación de maquinaria": "Manufacturing",
    "Fabricación de productos electrónicos": "Manufacturing",
    "Manufactura de productos para el cuidado de la salud": "Manufacturing",
    "Consumer Products Manufacturing": "Manufacturing",
    "Industrial Manufacturing": "Manufacturing",
    "Transportation Equipment Manufacturing": "Manufacturing",

    #Educacion
    "Universidades y escuelas profesionales": "Education",


    "-1": "Unspecified"
    }

    # Normalizar la industria (eliminar espacios, convertir a minúsculas)
    normalized_industry = industry.lower().strip()

    # Buscar la categoría correspondiente en el diccionario
    category = mapping.get(normalized_industry)
    return category

# Aplicar la función de mapeo a toda la columna 'type'
df['category'] = df['type'].apply(map_industry_to_category)

print(df)

                                       type   id    category
0                               Unspecified    1        None
1    Aseguradoras y sociedades de inversión    2     Finance
2              Banca y Servicios de crédito    3        None
3                    Desarrollo de software    4  Technology
4         Fabricación de productos químicos    5        None
..                                      ...  ...         ...
128                           Manufacturing  129        None
129                      Telecommunications  130        None
130                                  Retail  131        None
131                             Agriculture  132        None
132                                  Travel  133        None

[133 rows x 3 columns]


In [14]:
df.category.value_counts()

category
other          132
Unspecified      1
Name: count, dtype: int64

In [7]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet

# Descarga los recursos de NLTK si es necesario
nltk.download('punkt')
nltk.download('wordnet')

# Crear un DataFrame a partir de los datos (reemplaza con tus datos)
df=industry_type.copy()
df['type'] = df['type'].astype(str)

# Función para normalizar y stemmizar el texto
def normalize_text(text):
    stemmer = PorterStemmer()
    words = nltk.word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)


# Función para encontrar sinónimos y expandir el diccionario
def expand_mapping(mapping, word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    for synonym in synonyms:
        mapping[synonym] = mapping.get(word, word)
    return mapping

# Aplicar la función de normalización a la columna 'type'
df['type_normalized'] = df['type'].apply(normalize_text)


# Crear un diccionario de mapeo inicial (expandirlo según tus necesidades)
mapping = {
    'asegur': 'insurance',
    'banc': 'finance',
    'desarroll softwar': 'technology',
    'comput': 'technology',
    'salud': 'healthcare',
    
}

# Expandir el diccionario utilizando sinónimos
for index, row in df.iterrows():
    mapping = expand_mapping(mapping, row['type_normalized'])

# Función para mapear las categorías
def map_category(category):
    return mapping.get(category, 'other')

# Aplicar la función de mapeo

df['category'] = df['type_normalized'].apply(map_category)

# Eliminar duplicados basados en la categoría normalizada
df = df.drop_duplicates(subset='type_normalized', keep='first')

# Mostrar el DataFrame resultante
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daine\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\daine\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,type,id,type_normalized,category
0,Unspecified,1,unspecifi,other
1,Aseguradoras y sociedades de inversión,2,aseguradora y sociedad de inversión,other
2,Banca y Servicios de crédito,3,banca y servicio de crédito,other
3,Desarrollo de software,4,desarrollo de softwar,other
4,Fabricación de productos químicos,5,fabricación de producto químico,other


In [8]:
df.category.value_counts()

category
other        121
legal          1
consult        1
internet       1
mine           1
lend           1
truck          1
construct      1
account        1
transport      1
govern         1
retail         1
travel         1
Name: count, dtype: int64

In [10]:
df.sample(20)

,type,id,type_normalized,category
28,Organismos nacionales,29,organismo nacional,other
62,Banks & Credit Unions,63,bank & credit union,other
41,Gestión de activos e Inversiones,42,gestión de activo e inversion,other
79,Industrial Manufacturing,80,industri manufactur,other
91,Tv Broadcast & Cable Networks,92,tv broadcast & cabl network,other
57,Health Care Services & Hospitals,58,health care servic & hospit,other
131,Agriculture,132,agricultur,other
27,Belleza y Bienestar,28,belleza y bienestar,other
8,Tiendas de muebles y de artículos para el hogar,9,tienda de muebl y de artículo para el hogar,other
73,Logistics & Supply Chain,74,logist & suppli chain,other


In [ ]:
job_postings.info()

In [ ]:
job_postings.work_type.value_counts()

In [ ]:
job_postings.seniority_level_id.value_counts()

In [ ]:
job_postings.industry_type_id.value_counts()

In [ ]:
job_postings.sector.value_counts()

In [ ]:
job_postings.position_type_id.value_counts()

In [ ]:
# Crear el histograma
sns.histplot(data=job_postings, x='avg_salary', bins=20, kde=True)

In [13]:
df=job_postings[['seniority_level_id','industry_type_id','position_type_id','location_id','avg_salary']].copy()

In [14]:
# Definición de los rangos (ejemplo personalizado)
rangos = [0, 75000, 100000, 150000, 200000, np.inf]  # Ajusta los rangos según tu análisis
labels = ['Menor a 75000$', 'Entre 75000$ y 100000$', 'Entre 100000$ y 150000$', 'Entre 150000$ y 200000$', 'Mayor a 200000$']

# Creación de la nueva columna
df['rango_salarial'] = pd.cut(df['avg_salary'], bins=rangos, labels=labels)

In [ ]:
df.rango_salarial.value_counts()

In [ ]:
corr = df.corr(method='pearson')
plt.figure(figsize=(10,8))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15},
           xticklabels= df.columns, 
           yticklabels= df.columns,
           cmap= 'coolwarm')
plt.show()

In [ ]:
#definimos las variables atributos y la variable a predecir 
#df = job_postings[['seniority_level_id','position_type_id','location_id','avg_salary']]

# Codificar las columnas categóricas usando One-Hot Encoding
#encoder = OneHotEncoder()
#encoded_features = encoder.fit_transform(df[['position', 'state', 'level','industry_type_id']]).toarray()

# Crear un nuevo DataFrame con las features codificadas y la target
#df_encoded = pd.DataFrame(encoded_features)
# Estandarizar la columna 'avg_salary'
scaler = StandardScaler()
df['salary'] = scaler.fit_transform(df['avg_salary'].values.reshape(-1, 1))
#df_encoded['salary'] = df['avg_salary']

# Dividir los datos en entrenamiento y prueba
X = df.drop(['salary','avg_salary','industry_type_id'], axis=1).values
y = df['salary'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear un modelo de Random Forest
model = RandomForestRegressor()

# Entrenar el modelo
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)
print("Mean Squared Error:", mse.round(2))
print("Root Mean Squared Error:", rmse.round(2))
print("R2",r2.round(2))



## Redes neuronales

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [43]:
# Unir los datasets por los IDs correspondientes
data = pd.merge(job_postings, locations, left_on='location_id', right_on='id', how='left')
# Eliminar las columnas duplicadas (opcional)
data = data.drop(['id_x', 'id_y'], axis=1)
data = pd.merge(data, position_types, left_on='position_type_id', right_on='id', how='left')
data = data.drop(['id'], axis=1)
data = pd.merge(data, seniority_levels, left_on='seniority_level_id', right_on='id', how='left')


In [ ]:
data.info()

In [ ]:
# Codificar las columnas categóricas usando One-Hot Encoding
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(data[['position', 'state', 'level']]).toarray()

# Crear un nuevo DataFrame con las features codificadas y la target
df_encoded = pd.DataFrame(encoded_features)
# Estandarizar la columna 'avg_salary'
scaler = StandardScaler()
df_encoded['salary'] = scaler.fit_transform(data['avg_salary'].values.reshape(-1, 1))


# Dividir los datos en entrenamiento y prueba
X = df_encoded.drop('salary', axis=1).values
y = df_encoded['salary'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Definir el modelo
class SalaryPredictor(nn.Module, BaseEstimator):
  def __init__(self, input_size, hidden_size, output_size):
    super(SalaryPredictor, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = self.fc2(x)
    return x

  # Implement the fit method for scikit-learn compatibility
  def fit(self, X, y):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)

    train_dataset = TensorDataset(X_tensor, y_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Train the model (same as before)
    for epoch in range(num_epochs):
      for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = self(inputs)  # Use self(inputs) for calling forward pass
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

      if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Hiperparámetros
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100

# Crear el modelo, la función de pérdida y el optimizador
model = SalaryPredictor(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Crear dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Evaluación
with torch.no_grad():
    y_pred = []
    for inputs, labels in test_loader:
        outputs = model(inputs)
        y_pred.extend(outputs.numpy().flatten())

y_pred = np.array(y_pred)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MSE:",mse.round(2))
print("RMSE:", rmse.round(2))
print("R2:", r2.round(2))